## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pringsewu,ID,-5.3582,104.9744,74.12,93,100,0.89,overcast clouds
1,1,Broken Hill,AU,-31.9500,141.4333,80.01,28,72,14.70,broken clouds
2,2,Hilo,US,19.7297,-155.0900,71.11,82,20,6.91,few clouds
3,3,Tuktoyaktuk,CA,69.4541,-133.0374,-34.60,67,40,5.75,scattered clouds
4,4,Esperance,AU,-33.8667,121.9000,62.02,72,0,12.08,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Broken Hill,AU,-31.9500,141.4333,80.01,28,72,14.70,broken clouds
8,8,San Pedro,PH,14.3500,121.0167,79.36,92,75,5.75,broken clouds
11,11,Tete,MZ,-16.1564,33.5867,80.69,74,20,3.44,thunderstorm
14,14,Lagunas,PE,-5.2269,-75.6753,83.77,66,99,3.20,moderate rain
16,16,Alofi,NU,-19.0595,-169.9187,75.09,94,100,2.30,overcast clouds
17,17,Ndele,CF,8.4109,20.6473,82.62,14,85,5.32,overcast clouds
18,18,Atuona,PF,-9.8000,-139.0333,77.58,78,50,15.26,light rain
21,21,Porto Novo,BJ,6.4965,2.6036,84.02,67,60,9.80,broken clouds
23,23,Praya,CV,14.9215,-23.5087,77.54,61,0,18.41,clear sky
26,26,Georgetown,MY,5.4112,100.3354,80.89,87,40,8.05,scattered clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                216
City                   216
Country                211
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Current Description    216
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                211
City                   211
Country                211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Broken Hill,AU,80.01,broken clouds,-31.9500,141.4333,
8,San Pedro,PH,79.36,broken clouds,14.3500,121.0167,
11,Tete,MZ,80.69,thunderstorm,-16.1564,33.5867,
14,Lagunas,PE,83.77,moderate rain,-5.2269,-75.6753,
16,Alofi,NU,75.09,overcast clouds,-19.0595,-169.9187,
17,Ndele,CF,82.62,overcast clouds,8.4109,20.6473,
18,Atuona,PF,77.58,light rain,-9.8000,-139.0333,
21,Porto Novo,BJ,84.02,broken clouds,6.4965,2.6036,
23,Praya,CV,77.54,clear sky,14.9215,-23.5087,
26,Georgetown,MY,80.89,scattered clouds,5.4112,100.3354,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"     
    # 6d. Set up the base URL for the Google Directions API to get JSON data.    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.dropna()

In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))